<a href="https://colab.research.google.com/github/bennyqp/artificial-inspiration/blob/main/Image%20Generation/ai_image_generation_StyleTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Batch Style Transfer

This notebook lets you randomly apply image styles from images in one image dataset to images in a second dataset. 

_

This notebook is based on the 
#Original StyleTransfer Notebook by Derrick Schulz
You can find the original here:

https://github.com/dvschultz/ai/blob/master/neural_style_tf.ipynb

Also, this notebook uses a fork from the Neural Style Library with modifications by Derrick Schulz:

https://github.com/dvschultz/neural-style-tf

_

This notebook was modified for batch processing as part of the "Artificial Inspiration" project.

https://github.com/bennyqp/artificial-inspiration

##Set up our Runtime

We then need to make sure we’re using the latest version of Tensorflow 1, otherwise we get some annoying messages.

In [ ]:
#install TF 1.15 to avoid some annoying warning messages
# Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Install the neural-style-tf library

In [ ]:
#import some image display tools
from IPython.display import Image, display
#install the library in colab
!git clone https://github.com/bennyqp/neural-style-tf
#change into that directory
%cd neural-style-tf/
#install the library dependencies (it's likely Colab already has them installed, but let's be sure)
!pip install -r requirements.txt
#install the VGG19 pre-trained model
!wget http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat

In [ ]:
#let's make sure we're in the right folder
!pwd
# we should see "/content/neural-style-tf"

##Creat Hue Shift Function

For this project, we want to have as much variety and separation from the original images in the results as possible, so we create a function here that changes the hue of the style images before the actual style transfer.



In [4]:
from IPython.display import Image, display
from PIL import Image
import numpy as np
import colorsys

rgb_to_hsv = np.vectorize(colorsys.rgb_to_hsv)
hsv_to_rgb = np.vectorize(colorsys.hsv_to_rgb)

def shift_hue(arr, hout):
    r, g, b, a = np.rollaxis(arr, axis=-1)
    h, s, v = rgb_to_hsv(r, g, b)
    s = s * 1.0 ##########Sätigung erhöhen

    ###Manchmal random nur in einer Farbe?
    randomColor = random.randint(0, 100)
    if randomColor > 75:
      h = hout
    else:
      h = h + hout

    #h = h + hout  
    r, g, b = hsv_to_rgb(h, s, v)
    arr = np.dstack((r, g, b, a))
    return arr

def colorize(image, hue):
    img = image.convert('RGBA')
    arr = np.array(np.asarray(img).astype('float'))
    new_img = Image.fromarray(shift_hue(arr, hue/360.).astype('uint8'), 'RGBA')

    return new_img

##Input and Output Folder

Set the input and output folder and the folder where the style images are located.

The input folder should contain the files to which the StyleTrasfer will be applied. This can be one or more images. If there are multiple images, images will be randomly selected from the folder.

The output folder is where the final images are saved. 

The style images folder should contain the dataset of style images. The images can be located directly in the folder or they can be divided into subfolders. 

In [5]:
inputFolder = "/content/input"
outputFolder = "/content/output"
styleImages = "/content/styles"

In [ ]:
import os 

def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

styleImages = getListOfFiles(styleImages)
print("Number of Style Images: " + str(len(styleImages)))

##Set some parameters

In [7]:
# Basic parameters
imageNum = 500 #How many Images should be created?
imageSize = 256 #How big should the images be?
startCounter = 0 #With which number should the counter in the name of the generated images start? 


# Advanced parameters
iterationVariety = 500 # Variety of style intensity of the images. Recommended value between 400 and 1000
styleScaleVariety = 0.6 # Variety in the scaling of the style image. Recommended value between 0.3 and 1.0
oColorProbability = 1 # Probability from 0 to 100 with which the original color of the original image should be kept.
pickStyleImageRandomly = True # Should the style image be chosen randomly or according to the given order?
startAtStyleImage = 0 #If pickStyleImageRandomly == False: At which image should be started?


# If pickStyleImageRandomly == True: According to what probabilities should one, two, three or four images be used as style images. The probabilities are given between 0 and 1 (Should add up to 1).
if pickStyleImageRandomly == True:
  oneStyleProb = 0.9
  twoStyleProb = 0.05
  threeStyleProb = 0.03
  fourStlyeProb = 0.02
else:
  oneStyleProb = 1
  twoStyleProb = 0
  threeStyleProb = 0
  fourStlyeProb = 0

#Some initial processing. Do not change anything here.
startImageFolder = inputFolder
styleNumbers = len(styleImages)
startImages = os.listdir( startImageFolder )
startImageNumbers = len(startImages)

##Let's start the StyleTransfer!

In [ ]:
import random
import shutil

for i in range(imageNum):
  saveStepStylesDir = "/content/stepStyles/"
  !mkdir $saveStepStylesDir

  iterationNumber = 800 + random.randint(- (iterationVariety * 0.25) , iterationVariety * 0.75)
  styleScale = 0.7 + random.uniform(- (styleScaleVariety/2), styleScaleVariety/2)
  oColor = random.randint(0, 100)
  counter = startCounter + i
  randomHue1 = random.randint(0, 360)
  randomHue2 = random.randint(0, 360)
  randomHue3 = random.randint(0, 360)
  randomHue4 = random.randint(0, 360)

  startImageName = startImages[random.randint(0, startImageNumbers - 1)]

  if pickStyleImageRandomly == True:
    styleImage1 = styleImages[random.randint(0, styleNumbers - 1)]
    styleImage2 = styleImages[random.randint(0, styleNumbers - 1)]
    styleImage3 = styleImages[random.randint(0, styleNumbers - 1)]
    styleImage4 = styleImages[random.randint(0, styleNumbers - 1)]
  else:
    styleImage1 = styleImages[startAtStyleImage + i]

  stylesNumber = random.uniform(0.0, 1.0)

  if stylesNumber < oneStyleProb:
    print("number of Styles: 1")
    print("Input Image: " + startImageName)
    print("iterationNumber: " + str(iterationNumber))
    print("styleScale: " + str(styleScale))
    print("oColor: " + str(oColor))
    print("styleImage: " + styleImage1)
    print("randomHue1: " + str(randomHue1))

    style1 = Image.open(styleImage1)
    
    style1 = colorize(style1, randomHue1)
    style1.save(saveStepStylesDir + "stepStyle.PNG")

    if oColor < oColorProbability:
      !python neural_style.py --content_img $startImageName --style_imgs stepStyle.PNG --content_img_dir $startImageFolder --style_imgs_dir $saveStepStylesDir --max_size $imageSize --max_iterations $iterationNumber --style_scale $styleScale --img_output_dir $outputFolder --round_number $counter --original_colors
    else:
      !python neural_style.py --content_img $startImageName --style_imgs stepStyle.PNG --content_img_dir $startImageFolder --style_imgs_dir $saveStepStylesDir --max_size $imageSize --max_iterations $iterationNumber --style_scale $styleScale --img_output_dir $outputFolder --round_number $counter

  elif stylesNumber < oneStyleProb + twoStyleProb and stylesNumber >= oneStyleProb:
    print("number of Styles: 2")
    print("iterationNumber: " + str(iterationNumber))
    print("styleScale: " + str(styleScale))
    print("oColor: " + str(oColor))
    print("styleImage1: " + styleImage1)
    print("styleImage2: " + styleImage2)
    print("randomHue1: " + str(randomHue1))
    print("randomHue2: " + str(randomHue2))

    style1 = Image.open(styleImage1)
    style1 = colorize(style1, randomHue1)
    style1.save(saveStepStylesDir + styleImage1, "PNG")

    style2 = Image.open(styleImage2)
    style2 = colorize(style2, randomHue2)
    style2.save(saveStepStylesDir + styleImage2, "PNG")

    styleImgWeight1 = random.uniform(0.2, 0.8)
    styleImgWeight2 = 1 - styleImgWeight1
    print("style Image Weight 1: " + str(styleImgWeight1))
    print("style Image Weight 2: " + str(styleImgWeight2))

    if oColor < oColorProbability:
      !python neural_style.py --content_img $startImageName --style_imgs $styleImage1 $styleImage2 --style_imgs_weights $styleImgWeight1 $styleImgWeight2 --content_img_dir $startImageFolder --style_imgs_dir $saveStepStylesDir --max_size $imageSize --max_iterations $iterationNumber --style_scale $styleScale --img_output_dir $outputFolder --round_number $counter --original_colors
    else:
      !python neural_style.py --content_img $startImageName --style_imgs $styleImage1 $styleImage2 --style_imgs_weights $styleImgWeight1 $styleImgWeight2 --content_img_dir $startImageFolder --style_imgs_dir $saveStepStylesDir --max_size $imageSize --max_iterations $iterationNumber --style_scale $styleScale --img_output_dir $outputFolder --round_number $counter

  elif stylesNumber < oneStyleProb + twoStyleProb + threeStyleProb and stylesNumber >= oneStyleProb + twoStyleProb:
    print("number of Styles: 3")
    print("iterationNumber: " + str(iterationNumber))
    print("styleScale: " + str(styleScale))
    print("oColor: " + str(oColor))
    print("styleImage1: " + styleImage1)
    print("styleImage2: " + styleImage2)
    print("styleImage3: " + styleImage3)
    print("randomHue1: " + str(randomHue1))
    print("randomHue2: " + str(randomHue2))
    print("randomHue3: " + str(randomHue3))

    style1 = Image.open(styleImage1)
    style1 = colorize(style1, randomHue1)
    style1.save(saveStepStylesDir + styleImage1, "PNG")

    style2 = Image.open(styleImage2)
    style2 = colorize(style2, randomHue2)
    style2.save(saveStepStylesDir + styleImage2, "PNG")

    style3 = Image.open(styleImage3)
    style3 = colorize(style3, randomHue3)
    style3.save(saveStepStylesDir + styleImage3, "PNG")

    styleImgWeight1 = random.uniform(0.05, 0.45)
    styleImgWeight2 = random.uniform(0.05, styleImgWeight1)
    styleImgWeight3 = 1 - styleImgWeight1 - styleImgWeight2
    print("style Image Weight 1: " + str(styleImgWeight1))
    print("style Image Weight 2: " + str(styleImgWeight2))
    print("style Image Weight 3: " + str(styleImgWeight3))

    if oColor < oColorProbability:
      !python neural_style.py --content_img $startImageName --style_imgs $styleImage1 $styleImage2 $styleImage3 --style_imgs_weights $styleImgWeight1 $styleImgWeight2 $styleImgWeight3 --content_img_dir $startImageFolder --style_imgs_dir $saveStepStylesDir --max_size $imageSize --max_iterations $iterationNumber --style_scale $styleScale --img_output_dir $outputFolder --round_number $counter --original_colors
    else:
      !python neural_style.py --content_img $startImageName --style_imgs $styleImage1 $styleImage2 $styleImage3 --style_imgs_weights $styleImgWeight1 $styleImgWeight2 $styleImgWeight3 --content_img_dir $startImageFolder --style_imgs_dir $saveStepStylesDir --max_size $imageSize --max_iterations $iterationNumber --style_scale $styleScale --img_output_dir $outputFolder --round_number $counter

  else:
    print("number of Styles: 4")
    print("iterationNumber: " + str(iterationNumber))
    print("styleScale: " + str(styleScale))
    print("oColor: " + str(oColor))
    print("styleImage1: " + styleImage1)
    print("styleImage2: " + styleImage2)
    print("styleImage3: " + styleImage3)
    print("styleImage3: " + styleImage4)
    print("randomHue1: " + str(randomHue1))
    print("randomHue2: " + str(randomHue2))
    print("randomHue3: " + str(randomHue3))
    print("randomHue4: " + str(randomHue4))

    style1 = Image.open(styleImage1)
    style1 = colorize(style1, randomHue1)
    style1.save(saveStepStylesDir + styleImage1, "PNG")

    style2 = Image.open(styleImage2)
    style2 = colorize(style2, randomHue2)
    style2.save(saveStepStylesDir + styleImage2, "PNG")

    style3 = Image.open(styleImage3)
    style3 = colorize(style3, randomHue3)
    style3.save(saveStepStylesDir + styleImage3, "PNG")

    style4 = Image.open(styleImage4)
    style4 = colorize(style4, randomHue4)
    style4.save(saveStepStylesDir + styleImage4, "PNG")

    styleImgWeight1 = random.uniform(0.05, 0.35)
    styleImgWeight2 = random.uniform(0.05, 0.35)
    styleImgWeight3 = random.uniform(0.05, (1-styleImgWeight1-styleImgWeight2))
    styleImgWeight4 = 1 - styleImgWeight1 - styleImgWeight2 - styleImgWeight3
    print("style Image Weight 1: " + str(styleImgWeight1))
    print("style Image Weight 2: " + str(styleImgWeight2))
    print("style Image Weight 3: " + str(styleImgWeight3))

    if oColor < oColorProbability:
      !python neural_style.py --content_img $startImageName --style_imgs $styleImage1 $styleImage2 $styleImage3 $styleImage4 --style_imgs_weights $styleImgWeight1 $styleImgWeight2 $styleImgWeight3 --content_img_dir $startImageFolder --style_imgs_dir $saveStepStylesDir --max_size $imageSize --max_iterations $iterationNumber --style_scale $styleScale --img_output_dir $outputFolder --round_number $counter --original_colors
    else:
      !python neural_style.py --content_img $startImageName --style_imgs $styleImage1 $styleImage2 $styleImage3 $styleImage4 --style_imgs_weights $styleImgWeight1 $styleImgWeight2 $styleImgWeight3 --content_img_dir $startImageFolder --style_imgs_dir $saveStepStylesDir --max_size $imageSize --max_iterations $iterationNumber --style_scale $styleScale --img_output_dir $outputFolder --round_number $counter

  shutil.rmtree(saveStepStylesDir)

  print("Last Style Image: " + str(startAtStyleImage + i))
  print(" ")
  print("Done with this Image.")
  print(" ")
  print(" ")